In [1]:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
'''because of the version conflicts of sklearn and imblearn, I had to import six separately (because sklearn depreciated utils.six
in the version 0.23 and I am using 0.24). Imblearn gives a lot of conflicts with sklearn if you install any version lower than 0.8'''
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from imblearn.datasets import fetch_datasets
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn import FunctionSampler  # to use a idendity sampler

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StructuredDataset
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
#from aif360.metrics.utils import compute_boolean_conditioning_vector

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import Markdown, display

sns.set_context("poster")

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
    import load_preproc_data_adult, load_preproc_data_german, load_preproc_data_compas

In [2]:
## import dataset
dataset_used = "adult" # "adult", "german", "compas"
protected_attribute_used = 1 # 1, 2

if dataset_used == "adult":
#    dataset_orig = AdultDataset()
    dataset_orig = load_preproc_data_adult()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
    
elif dataset_used == "german":
    dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
        
    for i in range(1000):
        if (dataset_orig.labels[i] == 2.0):
            dataset_orig.labels[i] = 0
        else:
            dataset_orig.labels[i] = 1
        
    dataset_orig.favorable_label = 1
    dataset_orig.unfavorable_label = 0

    
elif dataset_used == "compas":
#     dataset_orig = CompasDataset()
    dataset_orig = load_preproc_data_compas()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]

In [3]:
#transform aif dataset into pandas dataframe to apply stratified train-test split
dataset_pd = dataset_orig.convert_to_dataframe()
dataset_df = dataset_pd[0]
dataset_df

,race,sex,Age (decade)=10,Age (decade)=20,Age (decade)=30,Age (decade)=40,Age (decade)=50,Age (decade)=60,Age (decade)=>=70,Education Years=6,Education Years=7,Education Years=8,Education Years=9,Education Years=10,Education Years=11,Education Years=12,Education Years=<6,Education Years=>12,Income Binary
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
48838,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
48839,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
48840,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
X = dataset_df.loc[:, dataset_df.columns != 'Income Binary']
y = dataset_df.loc[:,dataset_df.columns == 'Income Binary'].values
y = y.flatten()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=y)
type(y_train)
#will be used later
y_train_pd = pd.DataFrame(y_train)
type(y_train_pd)
y_test_pd = pd.DataFrame(y_test)
type(y_test_pd)

pandas.core.frame.DataFrame

In [6]:
#Class counts
[print('Class {} has {} instances in train set'.format(label, count))
 for label, count in zip(*np.unique(y_train, return_counts=True))]

#Class counts 2
[print('Class {} has {} instances in test set'.format(label, count))
 for label, count in zip(*np.unique(y_test, return_counts=True))]

Class 0.0 has 24894 instances in train set
Class 1.0 has 7830 instances in train set
Class 0.0 has 12261 instances in test set
Class 1.0 has 3857 instances in test set


[None, None]

In [7]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

num_pipe = make_pipeline(
    StandardScaler(), SimpleImputer(strategy="mean", add_indicator=True)
)

index = []
scores = {"Accuracy": [], "Balanced accuracy": []}


ind = []
results = {"AEO Difference": [], "Disparate Impact Ratio": [], "Dem Parity Difference": [], "Predictive Parity Difference": [],
           "Consistency": [],  "Accuracy": [], "Balanced accuracy": [],  "F1-Score": []}

In [8]:
#Uniting features and labels to use in fairness metrics
train_set = X_train.copy()
train_set = train_set.assign(labels = y_train_pd.values)
train_set

test_set = X_test.copy()
test_set = test_set.assign(labels = y_test_pd.values)
test_set

,race,sex,Age (decade)=10,Age (decade)=20,Age (decade)=30,Age (decade)=40,Age (decade)=50,Age (decade)=60,Age (decade)=>=70,Education Years=6,Education Years=7,Education Years=8,Education Years=9,Education Years=10,Education Years=11,Education Years=12,Education Years=<6,Education Years=>12,labels
24250,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7316,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2358,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
36905,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7065,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8512,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
13915,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5703,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
20931,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
#transform training set dataframe back to aif dataset type to calculate fairness metrics
aif_data = StructuredDataset(df=train_set, label_names=['labels'], protected_attribute_names=['sex'])
print(type(aif_data))

aif_train_binary = BinaryLabelDataset(df=train_set, label_names=['labels'], protected_attribute_names=['sex'])
print(type(aif_train_binary))
print(aif_train_binary.protected_attribute_names)

aif_test_binary = BinaryLabelDataset(df=test_set, label_names=['labels'], protected_attribute_names=['sex'])
print(type(aif_test_binary))
print(aif_test_binary.protected_attribute_names)

<class 'aif360.datasets.structured_dataset.StructuredDataset'>
<class 'aif360.datasets.binary_label_dataset.BinaryLabelDataset'>
['sex']
<class 'aif360.datasets.binary_label_dataset.BinaryLabelDataset'>
['sex']


In [10]:
#Logistic Regression Training for each dataset
log_reg = LogisticRegression() 

#Fitting the training set
log_reg.fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)

display(Markdown("#### LR predictions of stratified %s Test Set: Fairness Performance Results"%dataset_used))

#Create a new version of the test set with predicted class labels
testset_pred = aif_test_binary.copy()
testset_pred.labels = y_test_pred

#Construction 1
#to construct this metric function, the predicted labels should be united with the test fetures to make a new datas
metric_pred_test = BinaryLabelDatasetMetric(testset_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
#Construction 2
#both original test dataset with actual labels and the test dataset combined with predicted class labels need to be given to this function
classified_metric = ClassificationMetric(aif_test_binary, 
                                                 testset_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)


#Checking Equalized Odds: average odds differecence, which is the avg. of differences in FPR&TPR for privileged and unprivileged groups.
aeo = classified_metric.average_odds_difference()

#Disparate Impact ratio between privileged and unprivileged groups.
di = classified_metric.disparate_impact()

#Demographic parity difference between privileged and unprivileged groups.
spd = classified_metric.statistical_parity_difference()

#Predictive parity difference: PPV difference between privileged and unprivileged groups.
ppd = classified_metric.positive_predictive_value(privileged=False) - classified_metric.positive_predictive_value(privileged=True)

#Individual Fairness: 1)Consistency, 2) Euclidean Distance between individuals.
consistency = metric_pred_test.consistency()

TPR = classified_metric.true_positive_rate() #recall
TNR = classified_metric.true_negative_rate() #specificity
PPV = classified_metric.positive_predictive_value() #precision
bal_acc = (TPR+TNR)/2
f1 = 2*((PPV*TPR)/(PPV+TPR))
acc = classified_metric.accuracy()

ind += ["Log reg imb test set"]
results["AEO Difference"].append(aeo)
results["Disparate Impact Ratio"].append(di)
results["Dem Parity Difference"].append(spd)
results["Predictive Parity Difference"].append(ppd)
results["Consistency"].append(consistency)
results["Accuracy"].append(acc)
results["Balanced accuracy"].append(bal_acc)
results["F1-Score"].append(f1)
          
df_results = pd.DataFrame(results, index=ind)
    
display(Markdown("#### LR Prediction Performance on stratified %s Test Set"%dataset_used))

print("For %s dataset"%dataset_used)
print("Precision (PPV): %f" %PPV)
print("Recall (TPR): %f" %TPR)
print("Specificity (TNR): %f" %TNR)

#### LR predictions of stratified adult Test Set: Fairness Performance Results

c:\users\hatta\anaconda3\envs\aif360\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


#### LR Prediction Performance on stratified adult Test Set

For adult dataset
Precision (PPV): 0.650331
Recall (TPR): 0.381903
Specificity (TNR): 0.935405


In [11]:
#Random forest Training for each dataset
rfc = RandomForestClassifier(random_state=42, n_jobs=2)

#Fitting the training set
rfc.fit(X_train, y_train)
y_test_pred = rfc.predict(X_test)

display(Markdown("#### RF predictions of stratified %s Test Set: Fairness Performance Results"%dataset_used))

#Create a new version of the test set with predicted class labels
testset_pred = aif_test_binary.copy()
testset_pred.labels = y_test_pred

#Construction 1
#to construct this metric function, the predicted labels should be united with the test fetures to make a new datas
metric_pred_test = BinaryLabelDatasetMetric(testset_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
#Construction 2
#both original test dataset with actual labels and the test dataset combined with predicted class labels need to be given to this function
classified_metric = ClassificationMetric(aif_test_binary, 
                                                 testset_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)


#Checking Equalized Odds: average odds differecence, which is the avg. of differences in FPR&TPR for privileged and unprivileged groups.
aeo = classified_metric.average_odds_difference()

#Disparate Impact ratio between privileged and unprivileged groups.
di = classified_metric.disparate_impact()

#Demographic parity difference between privileged and unprivileged groups.
spd = classified_metric.statistical_parity_difference()

#Predictive parity difference: PPV difference between privileged and unprivileged groups.
ppd = classified_metric.positive_predictive_value(privileged=False) - classified_metric.positive_predictive_value(privileged=True)

#Individual Fairness: 1)Consistency, 2) Euclidean Distance between individuals.
consistency = metric_pred_test.consistency()

TPR = classified_metric.true_positive_rate() #recall
TNR = classified_metric.true_negative_rate() #specificity
PPV = classified_metric.positive_predictive_value() #precision
bal_acc = (TPR+TNR)/2
f1 = 2*((PPV*TPR)/(PPV+TPR))
acc = classified_metric.accuracy()

ind += ["Random F imb test set"]
results["AEO Difference"].append(aeo)
results["Disparate Impact Ratio"].append(di)
results["Dem Parity Difference"].append(spd)
results["Predictive Parity Difference"].append(ppd)
results["Consistency"].append(consistency)
results["Accuracy"].append(acc)
results["Balanced accuracy"].append(bal_acc)
results["F1-Score"].append(f1)
          
df_results = pd.DataFrame(results, index=ind)
    
display(Markdown("#### RF Prediction Performance on %s Test Set"%dataset_used))

print("For %s dataset"%dataset_used)
print("Precision (PPV): %f" %PPV)
print("Recall (TPR): %f" %TPR)
print("Specificity (TNR): %f" %TNR)

#### RF predictions of stratified adult Test Set: Fairness Performance Results

c:\users\hatta\anaconda3\envs\aif360\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


#### RF Prediction Performance on adult Test Set

For adult dataset
Precision (PPV): 0.641058
Recall (TPR): 0.389422
Specificity (TNR): 0.931409


In [12]:
df_results

,AEO Difference,Disparate Impact Ratio,Dem Parity Difference,Predictive Parity Difference,Consistency,Accuracy,Balanced accuracy,F1-Score
Log reg imb test set,-0.277990,0.0,-0.209335,-0.650331,0.999926,0.802953,0.658654,0.481215
Random F imb test set,-0.285674,0.0,-0.216543,-0.641058,0.999491,0.801712,0.660415,0.484516


In [13]:
#The oversampling must be done after the dataset is split into train and test sets!
#it cannot provide cluster balance with 0.5
sm = KMeansSMOTE(k_neighbors=2, random_state=42, )
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

#Class counts
[print('Class {} has {} instances'.format(label, count))
 for label, count in zip(*np.unique(y_train_res, return_counts=True))]

sm.get_params()

Class 0.0 has 24894 instances
Class 1.0 has 24894 instances


{'cluster_balance_threshold': 'auto',
 'density_exponent': 'auto',
 'k_neighbors': 2,
 'kmeans_estimator': None,
 'n_jobs': None,
 'random_state': 42,
 'sampling_strategy': 'auto'}

In [14]:
X_res_p = pd.DataFrame(X_train_res)

X_res_pd = pd.DataFrame(data=X_res_p.values, columns=dataset_orig.feature_names)
X_res_pd['labels'] = y_train_res

feature_names=X_res_pd.columns

#Check the ratio of class imbalance
y_res_p = pd.DataFrame(y_train_res)
y_res_p.value_counts(normalize=True)

1.0    0.5
0.0    0.5
dtype: float64

In [15]:
#transform the oversampled pandas dataframe object (training dataset) to aif dataset object
aif_data = StructuredDataset(df=X_res_pd, label_names=['labels'], protected_attribute_names=['sex'])

print(type(aif_data))

aif_binary = BinaryLabelDataset(df=X_res_pd, label_names=['labels'], protected_attribute_names=['sex'])
print(type(aif_binary))
print(aif_binary.protected_attribute_names)

<class 'aif360.datasets.structured_dataset.StructuredDataset'>
<class 'aif360.datasets.binary_label_dataset.BinaryLabelDataset'>
['sex']


In [16]:
# Initial disparities in the oversampled datasets

metric_transf = BinaryLabelDatasetMetric(aif_binary, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)

display(Markdown("#### Original %s train set"%dataset_used))
print("Disparate impact ratio (of transformed labels) between unprivileged and privileged groups = %f" % metric_transf.disparate_impact())
print("Difference in statistical parity (of transformed labels) between unprivileged and privileged groups = %f" % metric_transf.statistical_parity_difference())
print("Individual fairness metric 'consistency' that measures how similar the labels are for similar instances = %f" % metric_transf.consistency())

#### Original adult train set

Disparate impact ratio (of transformed labels) between unprivileged and privileged groups = 0.449435
Difference in statistical parity (of transformed labels) between unprivileged and privileged groups = -0.322211


c:\users\hatta\anaconda3\envs\aif360\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


Individual fairness metric 'consistency' that measures how similar the labels are for similar instances = 0.626014


In [17]:
#Training the oversampled dataset with logistic regression
#Logistic Regression Training for each dataset
log_reg_t = LogisticRegression() 

#Fitting the training set
log_reg_t.fit(X_train_res, y_train_res)
y_transf_test_pred = log_reg_t.predict(X_test)

display(Markdown("#### LR predictions of stratified %s Test Set: Fairness Performance Results"%dataset_used))

#Create a new version of the test set with predicted class labels
testset_transf_pred = aif_test_binary.copy()
testset_transf_pred.labels = y_transf_test_pred

#Construction 1
#to construct this metric function, the predicted labels should be united with the test fetures to make a new datas
metric_pred_test = BinaryLabelDatasetMetric(testset_transf_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
#Construction 2
#both original test dataset with actual labels and the test dataset combined with predicted class labels need to be given to this function
classified_metric = ClassificationMetric(aif_test_binary, 
                                                 testset_transf_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)


#Checking Equalized Odds: average odds differecence, which is the avg. of differences in FPR&TPR for privileged and unprivileged groups.
aeo = classified_metric.average_odds_difference()

#Disparate Impact ratio between privileged and unprivileged groups.
di = classified_metric.disparate_impact()

#Demographic parity difference between privileged and unprivileged groups.
spd = classified_metric.statistical_parity_difference()

#Predictive parity difference: PPV difference between privileged and unprivileged groups.
ppd = classified_metric.positive_predictive_value(privileged=False) - classified_metric.positive_predictive_value(privileged=True)

#Individual Fairness: 1)Consistency, 2) Euclidean Distance between individuals.
consistency = metric_pred_test.consistency()

TPR = classified_metric.true_positive_rate() #recall
TNR = classified_metric.true_negative_rate() #specificity
PPV = classified_metric.positive_predictive_value() #precision
bal_acc = (TPR+TNR)/2
f1 = 2*((PPV*TPR)/(PPV+TPR))
acc = classified_metric.accuracy()

ind += ["Log reg (oversampled) test set"]
results["AEO Difference"].append(aeo)
results["Disparate Impact Ratio"].append(di)
results["Dem Parity Difference"].append(spd)
results["Predictive Parity Difference"].append(ppd)
results["Consistency"].append(consistency)
results["Accuracy"].append(acc)
results["Balanced accuracy"].append(bal_acc)
results["F1-Score"].append(f1)
          
df_results = pd.DataFrame(results, index=ind)
    
display(Markdown("#### LR Prediction Performance on stratified %s Test Set"%dataset_used))

print("For %s dataset"%dataset_used)
print("Precision (PPV): %f" %PPV)
print("Recall (TPR): %f" %TPR)
print("Specificity (TNR): %f" %TNR)

#### LR predictions of stratified adult Test Set: Fairness Performance Results

c:\users\hatta\anaconda3\envs\aif360\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


#### LR Prediction Performance on stratified adult Test Set

For adult dataset
Precision (PPV): 0.559443
Recall (TPR): 0.509982
Specificity (TNR): 0.873664


In [18]:
#Random forest classifier after oversampling
rfc = RandomForestClassifier(random_state=42, n_jobs=2)
rfc.fit(X_train_res, y_train_res)
y_test_pred = rfc.predict(X_test)

display(Markdown("#### RFC predictions of %s Test Set: Fairness Performance Results"%dataset_used))

#Create a new version of the test set with predicted class labels
testset_transf_pred = aif_test_binary.copy()
testset_transf_pred.labels = y_test_pred

#Construction 1
#to construct this metric function, the predicted labels should be united with the test fetures to make a new datas
metric_pred_test = BinaryLabelDatasetMetric(testset_transf_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
#Construction 2
#both original test dataset with actual labels and the test dataset combined with predicted class labels need to be given to this function
classified_metric = ClassificationMetric(aif_test_binary, 
                                                 testset_transf_pred,
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)


#Checking Equalized Odds: average odds differecence, which is the avg. of differences in FPR&TPR for privileged and unprivileged groups.
aeo = classified_metric.average_odds_difference()

#Disparate Impact ratio between privileged and unprivileged groups.
di = classified_metric.disparate_impact()

#Demographic parity difference between privileged and unprivileged groups.
spd = classified_metric.statistical_parity_difference()

#Predictive parity difference: PPV difference between privileged and unprivileged groups.
ppd = classified_metric.positive_predictive_value(privileged=False) - classified_metric.positive_predictive_value(privileged=True)

#Individual Fairness: 1)Consistency, 2) Euclidean Distance between individuals.
consistency = metric_pred_test.consistency()

TPR = classified_metric.true_positive_rate() #recall
TNR = classified_metric.true_negative_rate() #specificity
PPV = classified_metric.positive_predictive_value() #precision
bal_acc = (TPR+TNR)/2
f1 = 2*((PPV*TPR)/(PPV+TPR))
acc = classified_metric.accuracy()

ind += ["Random F (oversampled) test set"]
results["AEO Difference"].append(aeo)
results["Disparate Impact Ratio"].append(di)
results["Dem Parity Difference"].append(spd)
results["Predictive Parity Difference"].append(ppd)
results["Consistency"].append(consistency)
results["Accuracy"].append(acc)
results["Balanced accuracy"].append(bal_acc)
results["F1-Score"].append(f1)
          
df_results = pd.DataFrame(results, index=ind)
    
display(Markdown("#### RF Prediction Performance on %s Test Set"%dataset_used))

print("For %s dataset"%dataset_used)
print("Precision (PPV): %f" %PPV)
print("Recall (TPR): %f" %TPR)
print("Specificity (TNR): %f" %TNR)

#### RFC predictions of adult Test Set: Fairness Performance Results

c:\users\hatta\anaconda3\envs\aif360\lib\site-packages\sklearn\utils\validation.py:72: FutureWarning: Pass n_neighbors=5 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


#### RF Prediction Performance on adult Test Set

For adult dataset
Precision (PPV): 0.600296
Recall (TPR): 0.421312
Specificity (TNR): 0.911753


In [19]:
df_results

,AEO Difference,Disparate Impact Ratio,Dem Parity Difference,Predictive Parity Difference,Consistency,Accuracy,Balanced accuracy,F1-Score
Log reg imb test set,-0.277990,0.000000,-0.209335,-0.650331,0.999926,0.802953,0.658654,0.481215
Random F imb test set,-0.285674,0.000000,-0.216543,-0.641058,0.999491,0.801712,0.660415,0.484516
Log reg (oversampled) test set,-0.165325,0.340154,-0.183805,-0.225039,0.999851,0.786636,0.691823,0.533568
Random F (oversampled) test set,-0.155666,0.320306,-0.146995,-0.306730,0.999702,0.794391,0.666532,0.495125
